In [6]:
# hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# dst = cv2.calcBackProject([hsv], [0,1], hist, [0,180,0,256], 1)

# disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
# cv2.filter2D(dst, -1, disc, dst)

# ret, thresh = cv2.threshold(dst, 100, 255, 0)
# thresh = cv2.merge((thresh,thresh, thresh))

# cv2.GaussianBlur(dst, (3,3), 0, dst)

# res = cv2.bitwise_and(img, thresh)

In [1]:
import cv2
import numpy as np

In [5]:
cap = cv2.VideoCapture(0)
# cap.set(3, 64)
# cap.set(4, 64)

while True:
    _, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
 
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_color = np.array([3, 50, 50])
    upper_color = np.array([33,255,255])

    # frame = imutils.resize(frame, width = 400)
    # converted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # skinMask = cv2.inRange(converted, lower, upper)

    mask = cv2.inRange(hsv, lower_color, upper_color)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    skinMask = cv2.erode(mask, kernel, iterations = 2)
    skinMask = cv2.dilate(mask, kernel, iterations = 2)
    skinMask = cv2.GaussianBlur(mask, (3, 3), 0)
    # skin = cv2.bitwise_and(frame, frame, mask = skinMask)
        # mask will be everything in range

    result = cv2.bitwise_and(frame, frame, mask = skinMask) 
    
    result = cv2.cvtColor(result, cv2.COLOR_HSV2BGR)

    
    ret, thresh = cv2.threshold(result, 75,255,cv2.THRESH_BINARY_INV)
#     _, thresh1 = cv2.threshold(blur, 240,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#     ret,thresh = cv2.threshold(img,127,255,0)
    
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    
#     thresh1 = cv2.erode(closing,None,iterations = 2)
    cv2.convertScaleAbs(closing, closing)
#     print(closing.shape)
    
    gray1 = cv2.cvtColor(closing, cv2.COLOR_BGR2GRAY)

    _,contours, hierarchy = cv2.findContours(gray1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
#     cv2.drawContours(frame, contours, -1, (0,255,0), 3)
    
    max_area = -11111
    second_max_area = -11111
    max_ci = -1
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        
        if area < max_area and area > second_max_area:
            second_max_area = area
            second_max_ci = i
        elif area>max_area:
            second_max_area = max_area
            max_area=area
            second_max_ci = max_ci
            max_ci = i
    cnt=contours[second_max_ci]
#     print(cnt)

#     for cnt in contours:
#             [x, y, w, h] = cv2.boundingRect(cnt)
#             cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 10)
    
    [x, y, w, h] = cv2.boundingRect(cnt)
#     hull = cv2.convexHull(cnt)
    
    if x < 50:
        x = 0
    else:
        x = x-50
        
    if y < 50:
        y=0
    else:
        y = y-50
        
    if x+w+50 > frame.shape[0]:
        w=frame.shape[0]
    else:
        w=x+w+50
    
    if y+h+50 > frame.shape[1]:
        h=frame.shape[1]
    else:
        h=y+h+50
    
    cv2.rectangle(frame, (x, y), (w, h), (255,0,0), 10)
    
    wanted_area = gray[x:w, y:h]
    
#     drawing = np.zeros(frame.shape,np.uint8)
#     cv2.drawContours(drawing,[cnt],0,(0,255,0),2)
#     cv2.drawContours(drawing,[hull],0,(0,0,255),2)
    
    
    cv2.imshow('frame', frame)
#     cv2.imshow('drawing', drawing)
    cv2.imshow('gray', wanted_area)
#     cv2.imshow('blur', blur)
#     cv2.imshow('closing', closing)
#     cv2.imshow('result', result)

    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 

error: D:\Build\OpenCV\opencv-3.4.0\modules\highgui\src\window.cpp:339: error: (-215) size.width>0 && size.height>0 in function cv::imshow
